### Google Maps API Script 2 - Obtaining the cuisines, ratings, and the price level.

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import json
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import scipy
import sys
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import emoji
import requests
import googlemaps
import warnings

In [15]:
# reading in api key and loading this in
f = open("api.txt", "r")
api_key = f.readlines()[1]
gmaps = googlemaps.Client(key=api_key, base_url='https://places.googleapis.com')

f.close()

In [3]:
restaurant_IDs = pd.read_csv('Restaurant_Location_Details.csv')

In [ ]:
cocotte_test = gmaps.place(place_id = 'ChIJJeZ74wEQdkgRRaSxXA9ggsM',
                                fields = ['price_level','rating','user_ratings_total','editorial_summary'], #,'primaryTypeName','primaryTypeDisplayName'
                                language = 'en',
                                reviews_no_translations = False, # translating reviews into English here for now
                                reviews_sort = 'newest')

In [16]:
brewery_test = gmaps.place(place_id = 'ChIJqdYI34sPdkgRvMVH8LGc4Mw',
                                fields = ['price_level','rating','user_ratings_total','editorial_summary'], #,'primaryTypeName','primaryTypeDisplayName'
                                language = 'en',
                                reviews_no_translations = False, # translating reviews into English here for now
                                reviews_sort = 'newest')

In [13]:
dict_list = [cocotte_test['result'], brewery_test['result']]

df = pd.DataFrame()
for i in range(2):
    dict_selected = dict_list[i]
    df = df.append(dict_selected, ignore_index=True)
print(df)



                                   editorial_summary  price_level  rating  \
0  {'language': 'en', 'overview': 'Welcoming, rus...            2     4.4   
1  {'language': 'en', 'overview': 'Gastropub serv...            2     4.2   

   user_ratings_total  
0                 740  
1                1619  


C:\Users\ndsna\AppData\Local\Temp\ipykernel_37436\2145559040.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict_selected, ignore_index=True)
C:\Users\ndsna\AppData\Local\Temp\ipykernel_37436\2145559040.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict_selected, ignore_index=True)


In [5]:
def extract_values(dictionary):
    overview_lang = dictionary['language']
    overview = dictionary['overview']
    return overview_lang, overview
df[['overview_lang', 'overview']] = df['editorial_summary'].apply(lambda x: pd.Series(extract_values(x)))

NameError: name 'df' is not defined

In [6]:
restaurant_list = restaurant_IDs['place_id']


In [20]:
def extract_values(dictionary):
    if dictionary == dictionary:
        overview_lang = dictionary['language']
        overview = dictionary['overview']
        return overview_lang, overview

missing_restaurants = []

with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)

    df = pd.DataFrame()
    for i in restaurant_list:
        restaurant_details = gmaps.place(place_id = i,
                                fields = ['price_level','rating','user_ratings_total','editorial_summary'], #,'primaryTypeName','primaryTypeDisplayName'
                                language = 'en',
                                reviews_no_translations = False, # translating reviews into English here for now
                                reviews_sort = 'newest')
        # moving past any case where it cannot be found
        if restaurant_details['result']:
            dict_details = restaurant_details['result']
            dict_details["place_id"] = i
            df = df.append(dict_details, ignore_index=True)

        # else:
        #     missing_restaurants = missing_restaurants.append(i)

    df[['overview_lang', 'overview']] = df['editorial_summary'].apply(lambda x: pd.Series(extract_values(x)))
    print(df)




                                      editorial_summary  price_level  rating  \
0     {'language': 'en', 'overview': 'Welcoming, rus...          2.0     4.4   
1     {'language': 'en', 'overview': 'A carefully-cu...          NaN     4.7   
2     {'language': 'en', 'overview': 'Charming Greek...          2.0     4.6   
3                                                   NaN          NaN     4.8   
4     {'language': 'en', 'overview': 'Inmates prep a...          2.0     4.8   
...                                                 ...          ...     ...   
1780  {'language': 'en', 'overview': 'A marble featu...          3.0     4.2   
1781  {'language': 'en', 'overview': 'Modern Italian...          2.0     3.8   
1782  {'language': 'en', 'overview': 'Hotel's upscal...          NaN     4.0   
1783  {'language': 'en', 'overview': 'Upbeat, party-...          2.0     4.2   
1784  {'language': 'en', 'overview': 'Gastropub serv...          2.0     4.2   

      user_ratings_total               

In [14]:
print(dict_details)

{'editorial_summary': {'language': 'en', 'overview': 'Sophisticated decor with tiles and exposed bricks in a daytime weekend cafe and nighttime bar.'}, 'price_level': 2, 'rating': 4.8, 'user_ratings_total': 826}


In [21]:
df.to_csv('Restaurant_Location_Additional_Details.csv', index = False)

In [49]:
# https://stackoverflow.com/questions/77890446/how-to-call-via-google-places-apinew-text-search-id-only-in-python

# Found that the Python library that interacts with the Google API doesn't seem to work with the 
# new version of the Places API, so adapted the code from the above to create my own function which
# will find the additional details I need, in particular it is the cuisine in order to create an
# intelligble recommendation list of similar restaurants

def find_place(places_id, api_key2):    
    # Define the API endpoint
    url = 'https://places.googleapis.com/v1/places/' + places_id

    # Define the headers
    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': api_key2, 
        'X-Goog-FieldMask': 'id,primaryType,location'
    }

    # Define the data payload for the POST request
    # data = {
    #     'textQuery': query
    # }

    # Make the POST request
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Error: {response.status_code}, {response.text}")
    return json.loads(response.text)



In [50]:
cocotte_test_custom = find_place(places_id='ChIJJeZ74wEQdkgRRaSxXA9ggsM', api_key2 = api_key)

In [42]:
restaurant_list = restaurant_IDs['place_id']

In [51]:
def extract_values(dictionary):
    if dictionary == dictionary:
        overview_lang = dictionary['latitude']
        overview = dictionary['longitude']
        return overview_lang, overview

missing_restaurants = []

with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)

    df = pd.DataFrame()
    for i in restaurant_list:
        restaurant_details = find_place(places_id=i, api_key2 = api_key)
        # moving past any case where it cannot be found
        if restaurant_details['id']:
            df = df.append(restaurant_details, ignore_index=True)

        # else:
        #     missing_restaurants = missing_restaurants.append(i)

    df[['latitude', 'longitude']] = df['location'].apply(lambda x: pd.Series(extract_values(x)))
    print(df)




                               id  \
0     ChIJJeZ74wEQdkgRRaSxXA9ggsM   
1     ChIJL5dqGYUbdkgRQ7cwcCeg_zw   
2     ChIJ-1A3NFMbdkgRAzDMdHvhCFI   
3     ChIJqdYI34sPdkgRvMVH8LGc4Mw   
4     ChIJw557YUcEdkgR7ANv_I8AvE4   
...                           ...   
1818  ChIJ4wPY0M0EdkgRLGC4aapz5gQ   
1819  ChIJ4fluy14FdkgR36zDvrWrWH4   
1820  ChIJG4wWn9YEdkgReoek5-tp5rA   
1821  ChIJW5xvHeAEdkgRywrT6Jn4zDo   
1822  ChIJGdIp0YQCdkgRvfDciFkSA_w   

                                               location         primaryType  \
0     {'latitude': 51.5151529, 'longitude': -0.19301...   brunch_restaurant   
1     {'latitude': 51.519576, 'longitude': -0.179230...  seafood_restaurant   
2     {'latitude': 51.593176799999995, 'longitude': ...    greek_restaurant   
3     {'latitude': 51.488687899999995, 'longitude': ...          restaurant   
4     {'latitude': 51.4519672, 'longitude': -0.12563...          restaurant   
...                                                 ...                 ...   
18

In [52]:
df.to_csv('Restaurant_Cuisine_and_Loc.csv', index = False)